# Lesson 2: Themes and Topics

Given two documents how can we determine what they are about? In this lab we will explore two methods for doing this.

The first uses statistics such as mutual information and chi-squared to determine unique words. 

The second uses a more advanced method called Latent Dirichlet Algorithm or LDA to determine which topics are present in the document.

## Corpus

The dataset is a text file with two columns. The first column contains one of three document names:
* "Quran"
* "OT"
* "NT"

where OT is Old Testament and NT is New Testament.
The second column contains the document text, in this case a verse from one of the three document names. 

**The document name and the document are separated by a tab character '\t'**.

Run the following code to output the first five lines of the dataset.

In [ ]:
with open("datasets/comparing_train_and_dev.tsv") as f:
    for i in range(5):
        print(f.readline())

## Preprocessing



In [ ]:
import re
import Stemmer
import os
from collections import Counter
import unittest
%reload_ext ipython_unittest

The cell below contains the preprocessing code for English text from Lab 1.

In [ ]:
# The file 'datasets/stopwords_en.txt' contains a list of stopwords - one per line.
with open("datasets/stopwords_en.txt") as f:
    enStopWords = set(f.read().splitlines())

#      stopWords.extend(['thy', 'ye', 'thou', 'thee', 'shalt', 'hath'])

# Initialze the SnowballStemmer
enStemmer = Stemmer.Stemmer('english')

def preprocess_line_en(line: str) -> list[str]:
    # Convert to lower case
    tokens = line.lower()
    
    # Split into tokens with no punctuation
    tokens = re.split("[^\w]", tokens)
    
    # Remove empty strings and stop words and apply the stemmer
    tokens = [enStemmer.stemWord(x) for x in tokens if x and x not in enStopWords]
    
    # Return the tokens
    return tokens

## Exercise 1

Given a doc_name "Quran", "NT", or "OT" and implement the following function that reads each line of the corpus and preprocesses the selected documents.

In [ ]:
def preprocess_dataset(doc_name: str, dataset: list[str]) -> list[str]:
    """Preprocesses and returns the documents from the dataset with the given doc name"""
    
    # 1. Loop over each line in the dataset and split it on the tab character '\t'. 
    # The first part is the document name the second part is the document.
    
    # 2. If the document name is the one specified in the parameter doc_name then preprocess it using 
    # preprocess_line_en.
    
    # 3. Return all the preprocessed documents.
    # Example return value: [['owner', 'day', 'recompens'], ['guid', 'straight', 'path'], ...]
    
    raise Exception("Implement me!")

Run the following test to verify your preprocessing function works as expected. If it passes you should see it succeeds

In [ ]:
test_corpus = ['Quran\tPraise be to Allah, Lord of the Worlds,',
 'Quran\tthe Merciful, the Most Merciful,',
 'Quran\tOwner of the Day of Recompense.',
 'Quran\tGuide us to the Straight Path,',
 'NT\tNow the birth of Jesus Christ was on this wise: When as his mother Mary was espoused to Joseph, before they came together, she was found with child of the Holy Ghost.',
 'NT\tThen Joseph her husband, being a just man, and not willing to make her a publick example, was minded to put her away privily.',
 'NT\tBut while he thought on these things, behold, the angel of the LORD appeared unto him in a dream, saying, Joseph, thou son of David, fear not to take unto thee Mary thy wife: for that which is conceived in her is of the Holy Ghost.',
 'NT\tAnd she shall bring forth a son, and thou shalt call his name JESUS: for he shall save his people from their sins.',
 'OT\tAnd the earth was without form, and void; and darkness was upon the face of the deep. And the Spirit of God moved upon the face of the waters.',
 'OT\tAnd God said, Let there be light: and there was light.',
 'OT\tAnd God saw the light, that it was good: and God divided the light from the darkness.',
 'OT\tAnd God called the light Day, and the darkness he called Night. And the evening and the morning were the first day.',
 'OT\tAnd God said, Let there be a firmament in the midst of the waters, and let it divide the waters from the waters.']

In [ ]:
%%unittest
"Quran documents are correctly processed"
result = preprocess_dataset("Quran", test_corpus)
assert result == [['prais', 'allah', 'lord', 'world'], ['merci', 'merci'], ['owner', 'day', 'recompens'], ['guid', 'straight', 'path']]

"Old Testament documents are correctly processed"
result = preprocess_dataset("OT", test_corpus)
assert result == [['earth', 'form', 'void', 'dark', 'face', 'deep', 'spirit', 'god', 'move', 'face', 'water'], ['god', 'light', 'light'], ['god', 'light', 'good', 'god', 'divid', 'light', 'dark'], ['god', 'call', 'light', 'day', 'dark', 'call', 'night', 'even', 'morn', 'day'], ['god', 'firmament', 'midst', 'water', 'divid', 'water', 'water']]

"New Testament documents are correctly processed"
result = preprocess_dataset("NT", test_corpus)
assert result == [['birth', 'jesus', 'christ', 'wise', 'mother', 'mari', 'espous', 'joseph', 'found', 'child', 'holi', 'ghost'], ['joseph', 'husband', 'man', 'make', 'publick', 'mind', 'put', 'privili'], ['thought', 'thing', 'behold', 'angel', 'lord', 'appear', 'dream', 'joseph', 'thou', 'son', 'david', 'fear', 'thee', 'mari', 'thi', 'wife', 'conceiv', 'holi', 'ghost'], ['bring', 'son', 'thou', 'shalt', 'call', 'jesus', 'save', 'peopl', 'sin']]

## Mutual Information & Chi Squared

The following functions in the cell below calculate mutual information and chi squared.

In [ ]:
from math import log2

In [ ]:
def compute_term_doc_freqs(corpus):
    # For each term find the number of docs in the corpus that contain it
    freqs = Counter()

    for doc in corpus:
        terms = set(doc)
        freqs.update(terms)
        
    return freqs 

def xlogy(x, y):
    # Compute x * log(y). If x == 0 then return zero without
    # evaluating log(y)
    if x == 0:
        return 0
    else:
        return x * log2(y)

def compute_mi(n00, n10, n01, n11):
    # Compute mutual information
    N = n00 + n10 + n01 + n11
    return xlogy((n11 / N), (N * n11) / ((n11 + n10) * (n11 + n01))) + \
            xlogy((n01 / N), (N * n01) / ((n01 + n00) * (n11 + n01))) + \
            xlogy((n10 / N), (N * n10) / ((n11 + n10) * (n10 + n00))) + \
            xlogy((n00 / N), (N * n00) / ((n01 + n00) * (n10 + n00)))

def compute_chi_sq(n00, n10, n01, n11):
    # Compute chi squared
    return ((n11 + n10 + n01 + n00) * (n11 * n00 - n10 * n01)**2) / \
            ((n11 + n01) * (n11 + n10) * (n10 + n00) * (n01 + n00))

def compute_mi_and_chi_sq(corpus, other_corpus, vocab):
    # Compute the term doc frequencies for the corpus and the other corpus
    corpus_freqs = compute_term_doc_freqs(corpus)
    other_corpus_freqs = compute_term_doc_freqs(other_corpus)

    # For each term in the vocab compute the MI and Chi squared
    mut_inf = {}
    chi_sq = {}

    for term in vocab:

        # Number of docs in corpus containing term
        n11 = corpus_freqs.get(term, 0)

        # Number of docs in corpus not containing term
        n10 = len(corpus) - n11 

        # Number of docs not in corpus containing term
        n01 = other_corpus_freqs.get(term, 0)

        # Number of docs not in corpus not containing term
        n00 = len(other_corpus) - n01 

        # Compute MI
        mut_inf[term] = compute_mi(n00, n10, n01, n11)

        # Compute Chi squared
        chi_sq[term] = compute_chi_sq(n00, n10, n01, n11)
    
    return mut_inf, chi_sq 

## Exercise 2

Write code following the steps below to calculate mutual information and chi squared for the corpus.

In [ ]:
def read_corpus() -> list[str]:
    with open("datasets/comparing_train_and_dev.tsv") as f:
        return f.readlines()

In [ ]:
# 1. Read the corpus by calling the function read_corpus defined above. 

# 2. Preprocess the datasets "Quran", "OT", and "NT" using the preprocess_dataset function.

# 3. Generate the vocabulary (a list of all the unique words) in the entire preprocessed corpus.

# 4. By calling the function compute_mi_and_chi_sq calculate the mutual information 
# and chi squared values for the Quran. The parameter corpus should be the preprocessed Quran documents and
# the 'other_corpus' is the combination of the preprocessed old and new testaments.

# 5. Sort the statistics from highest to lowest and print the top 10 mutual information words 
# and the top 10 chi squared words for the Quran.

# 6. Repeat steps 4 and 5 to calculate the mutual information and chi squared for the Old Testament and 
# the New Testament.


### Analysis

1. What differences do you observe between the top Mutual Information words and the top Chi Squared words?

2. What can you learn about the three documents, the Quran, Old Testament, and New Testament from these results?

## Extension exercises

1. Notice that words like "thou", "thi", "ye" appear in the output - what do you think these words mean in English?
2. It turns out these are old English words for "you" and "they" - but these should be stop words! Modify the preprocessing code to exclude these words.
3. Recalculate the Mutual Information and Chi Squared scores. Do you get better results?

## LDA

In [ ]:
import numpy as np
from gensim import models
from gensim.corpora.dictionary import Dictionary

In [ ]:
corpus = read_corpus()
quran = preprocess_dataset("Quran", corpus)
new_testament = preprocess_dataset("NT", corpus)
old_testament = preprocess_dataset("OT", corpus)

# Combine all the preprocessed corpora together
common_texts = quran + new_testament + old_testament

# Create dictionary 
corpus_dictionary = Dictionary(common_texts)
corpus = [corpus_dictionary.doc2bow(d) for d in common_texts]

In [ ]:
# Train the LDA model with 20 topics
lda = models.LdaModel(corpus, num_topics=20, id2word=corpus_dictionary)

In [ ]:
# Inspect the top 5 docs from the quran
for i in range(5):
    doc_bow = corpus_dictionary.doc2bow(quran[i])
    topics = lda.get_document_topics(doc_bow)
    print('Document: \n', quran[i])
    print('Topics: \n', topics)

In [ ]:
# Print out the top 5 words for each topic
print("The top five words for each topic:")
topics = lda.print_topics(num_words=5)
for topic_id, words in topics:
    print(topic_id, ': ', words)

## Exercise 3

In this exercise we'll write some code to interpret the topics identified by the LDA model.

1. For each of the three corpora: Quran, Old Testament, and New Testament, compute the average score for each topic by summing the document-topic probability for each document in that corpus and dividing by the total number of documents in the corpus.
2. Then for each corpus, identify the top topic that has the highest average score. For each of those three top topics print the top 10 tokens with highest probability of belonging to that topic. Hint: use `lda.print_topic`.

### Analysis

1. For each of the three top topics what title in 1-3 words would you give to that topic?

2. What does the LDA model tell you about the corpus? Consider the top three topics you have found. 

### Extension Exercises

1. Print out a list of all the topics for each corpus. Are there any topics that appear to be common in 2 corpora but not in the other? What are they and what are some examples of high probability words from these topics?

2. How do these results differ from what you learned using mutual information and chi squared?